# **Example inference pipeline**
This notebook infers lineages from the [Briney et al. 2019](https://doi.org/10.1038/s41586-019-0879-y) dataset.

##  1. Download data  
Annotated data can be downloaded using links provided in the [briney/grp_paper repository](https://github.com/briney/grp_paper).  
Uncomment following two lines to download all data in `./data_with_scripts/`  folder (make sure your current working directory is `.../HILARy/`)

In [1]:
#!wget http://burtonlab.s3.amazonaws.com/sequencing-data/hiseq_2016-supplement/316188_HNCHNBCXY_consensus_UID18-cdr3nt-90_minimal_071817.tar.gz
#!tar -xvf 316188_HNCHNBCXY_consensus_UID18-cdr3nt-90_minimal_071817.tar.gz --directory data_with_scripts/data_from_briney


consensus-cdr3nt-90_minimal/
consensus-cdr3nt-90_minimal/14_consensus.txt
consensus-cdr3nt-90_minimal/6_consensus.txt
consensus-cdr3nt-90_minimal/3_consensus.txt
consensus-cdr3nt-90_minimal/15_consensus.txt
consensus-cdr3nt-90_minimal/16_consensus.txt
consensus-cdr3nt-90_minimal/7_consensus.txt
consensus-cdr3nt-90_minimal/18_consensus.txt
consensus-cdr3nt-90_minimal/10_consensus.txt
consensus-cdr3nt-90_minimal/1_consensus.txt
consensus-cdr3nt-90_minimal/11_consensus.txt
consensus-cdr3nt-90_minimal/9_consensus.txt
consensus-cdr3nt-90_minimal/4_consensus.txt
consensus-cdr3nt-90_minimal/5_consensus.txt
consensus-cdr3nt-90_minimal/8_consensus.txt
consensus-cdr3nt-90_minimal/12_consensus.txt
consensus-cdr3nt-90_minimal/17_consensus.txt
consensus-cdr3nt-90_minimal/2_consensus.txt
consensus-cdr3nt-90_minimal/13_consensus.txt


## 2. Convert Briney data into airr format required by Hilary

### 2.1 install required libraries

In [2]:
#!pip install hilary==1.2.0
#!pip install biopython


You should consider upgrading via the '/home/gathenes/.pyenv/versions/3.9.6/envs/hilary/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/gathenes/.pyenv/versions/3.9.6/envs/hilary/bin/python3.9 -m pip install --upgrade pip' command.


### 2.2 Process briney data

In [1]:
import os
import pandas as pd
from tqdm import tqdm
from hilary.utils import create_classes


In [3]:
from compatible import Compatible
compatible = Compatible()
usecols = [
    "seq_id",
    "chain",
    "productive",
    "v_full",
    "j_full",
    "cdr3_nt",
    "v_start",
    "vdj_nt",
    "isotype",
]
dirname = "./data_with_scripts/data_from_briney/consensus-cdr3nt-90_minimal"
dfs = []
for filename in tqdm(os.listdir(dirname)):
    df = pd.read_csv(os.path.join(dirname, filename), usecols=usecols)
    dfs.append(compatible.df2airr(df))
df = pd.concat(dfs, ignore_index=True)
df["sequence_id"] = df.index
filename = "./data_with_scripts/data_from_briney/316188_ids.tsv.gz"
df[["seq_id", "sequence_id"]].to_csv(filename, sep="\t", index=False)
df.drop("seq_id", axis=1, inplace=True)
filename = "./data_with_scripts/data_from_briney/316188.tsv.gz"
usecols = [
    "sequence_id",
    "v_call",
    "j_call",
    "junction",
    "v_sequence_alignment",
    "j_sequence_alignment",
    "v_germline_alignment",
    "j_germline_alignment",
]
df[usecols].to_csv(filename, sep="\t", index=False)


  0%|          | 0/18 [00:00<?, ?it/s]

100%|██████████| 18/18 [04:16<00:00, 14.27s/it]


In [4]:
usecols = ['sequence_id',
        'v_call',
        'j_call',
        'junction',
        'v_sequence_alignment',
        'j_sequence_alignment',
        'v_germline_alignment',
        'j_germline_alignment']
filename = "data_with_scripts/data_from_briney/316188.tsv.gz"
dataframe = pd.read_table(filename,usecols=usecols)


In [5]:
print(dataframe.columns)


Index(['sequence_id', 'v_call', 'j_call', 'junction', 'v_sequence_alignment',
       'j_sequence_alignment', 'v_germline_alignment', 'j_germline_alignment'],
      dtype='object')


## 3. Package tutorial to infer lineages in python script

### 3.0 Uncomment next line to run on 100 000 sequences

In [9]:
#dataframe=dataframe.head(100000)


### 3.1 Create apriori object

In [6]:
from hilary.apriori import Apriori
apriori = Apriori(silent=False, threads=-1, precision=0.99, sensitivity=0.9) # show progress bars, use all threads


In [7]:
dataframe=dataframe.dropna()


In [8]:
dataframe_processed = apriori.preprocess(df=dataframe, df_kappa=None)
apriori.classes= create_classes(dataframe_processed)


100%|██████████| 7180/7180 [09:03<00:00, 13.21it/s]  


### 3.2 Infer histogram, parameters rho and mu, and sensitivity & precision thresholds for all classes

In [9]:
apriori.get_histograms(dataframe_processed)
apriori.get_parameters()
apriori.get_thresholds()


2024-03-08 04:48:38 [debug    ] Computing CDR3 hamming distances within all large VJl classes.


100%|██████████| 83/83 [00:00<00:00, 116.75it/s]


### 3.3 Create hilary object from apriori

In [11]:
from hilary.inference import HILARy
hilary=HILARy(apriori,df=dataframe_processed)


### 3.4 Compute precise and sensitive clusters

In [12]:
dataframe_cdr3=hilary.compute_prec_sens_clusters(df=dataframe_processed)


100%|██████████| 7180/7180 [00:08<00:00, 822.66it/s] 


### 3.5 Infer clonal families from these clusters

In [16]:
hilary.get_xy_thresholds(df=dataframe_cdr3)


  0%|          | 0/7263 [00:00<?, ?it/s]

100%|██████████| 7263/7263 [00:25<00:00, 279.99it/s]


In [17]:
hilary.classes["xy_threshold"] = hilary.classes["xy_threshold"]


In [18]:
dataframe_inferred = hilary.infer(df=dataframe_cdr3)


2024-03-08 06:01:07 [debug    ] Checking alignment length.     alignment_length=271
2024-03-08 06:01:08 [debug    ] Inferring family clusters for small groups.


100%|██████████| 4133/4133 [00:16<00:00, 256.90it/s]


2024-03-08 06:01:33 [debug    ] Inferring family clusters for large groups.


In [20]:
dataframe_inferred.to_csv(
    "./data_with_scripts/data_from_briney/briney_clonal_families.csv"
)
